In [2]:
import pandas as pd
import numpy as np

In [3]:
# Importing the Dataset
lisbon_coimbra_df = pd.read_excel('Lisbon_and_Coimbra_517_Proteomics.xlsx', sheet_name='517_Proteins')

# Generating the target array (y)
y = np.array([1 if i == 'Amyloid-Positive' else 0 for i in lisbon_coimbra_df['Class']]) # 1 -> Ab+      2 -> Ab-
print(f'- The number of Amyloid-Positive patients is: {list(y).count(1)} \n- The number of Amyloid-Negative patients is: {list(y).count(0)} \n')

# Dropping the useless columns 
lisbon_coimbra_df = lisbon_coimbra_df.drop(['Age', 'Gender', 'Sample code', 'Class'], axis=1)

# Generating the feature matrix (X)
X = np.array(lisbon_coimbra_df.values)
print(f'The feature matrix (X) is composed of: {X.shape[0]} examples and {X.shape[1]} features')

# Extracting the protein name
protein_code = list(lisbon_coimbra_df.columns)

- The number of Amyloid-Positive patients is: 67 
- The number of Amyloid-Negative patients is: 59 

The feature matrix (X) is composed of: 126 examples and 517 features


In [4]:
lisbon_coimbra_df

,ALBU_HUMAN,TRFE_HUMAN,TTHY_HUMAN,PTGDS_HUMAN,FBLN1_HUMAN,CO3_HUMAN,A2MG_HUMAN,FINC_HUMAN,CFAH_HUMAN,IGG1_HUMAN,...,SRPX_HUMAN,MDGA1_HUMAN,LV218_HUMAN,ACYP2_HUMAN,EGFLA_HUMAN,DSG2_HUMAN,CO6A2_HUMAN,LV223_HUMAN,MOES_HUMAN,CSPG4_HUMAN
0,0.356318,0.064232,0.015572,0.016359,0.004050,0.012248,0.004779,0.002982,0.002152,0.019001,...,0.000015,0.000067,0.000026,0.000028,0.000055,0.000042,0.000013,0.000043,7.468032e-06,0.000005
1,0.386206,0.047139,0.011901,0.012724,0.001745,0.018902,0.003778,0.001582,0.001820,0.039033,...,0.000036,0.000080,0.000034,0.000080,0.000057,0.000026,0.000016,0.000016,1.168286e-07,0.000010
2,0.487252,0.029375,0.019694,0.013325,0.002766,0.012882,0.003005,0.001655,0.001678,0.018865,...,0.000033,0.000031,0.000031,0.000078,0.000017,0.000038,0.000181,0.000005,2.735813e-06,0.000008
3,0.349113,0.071905,0.013097,0.016280,0.002784,0.018822,0.005564,0.002036,0.002687,0.032280,...,0.000064,0.000077,0.000046,0.000078,0.000007,0.000043,0.000025,0.000008,7.837693e-06,0.000011
4,0.326597,0.064845,0.018108,0.024601,0.002848,0.014416,0.004604,0.002868,0.001743,0.032797,...,0.000040,0.000077,0.000076,0.000028,0.000013,0.000049,0.000031,0.000060,1.470958e-05,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0.582099,0.034406,0.009165,0.015060,0.001440,0.009734,0.003096,0.001992,0.001210,0.017647,...,0.000195,0.000056,0.000032,0.000023,0.000015,0.000024,0.000022,0.000028,1.041795e-05,0.000007
122,0.405769,0.021734,0.006709,0.014386,0.000929,0.006573,0.001770,0.002009,0.001060,0.020147,...,0.000011,0.000013,0.000099,0.000018,0.000009,0.000018,0.000008,0.000016,1.983038e-05,0.000002
123,0.396349,0.019385,0.008458,0.019213,0.001399,0.006511,0.002858,0.002564,0.001268,0.034381,...,0.000023,0.000026,0.000120,0.000009,0.000045,0.000066,0.000147,0.000027,7.333960e-06,0.000003
124,0.343654,0.022957,0.013165,0.025490,0.001583,0.007964,0.003435,0.003775,0.001264,0.024775,...,0.000037,0.000036,0.000211,0.000049,0.000013,0.000036,0.000027,0.000019,1.735675e-05,0.000002


# ReliefF

In [5]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances


def reliefF(X, y, **kwargs):
    """
    This function implements the reliefF feature selection

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels
    kwargs: {dictionary}
        parameters of reliefF:
        k: {int}
            choices for the number of neighbors (default k = 5)

    Output
    ------
    score: {numpy array}, shape (n_features,)
        reliefF score for each feature

    Reference
    ---------
    Robnik-Sikonja, Marko et al. "Theoretical and empirical analysis of relieff and rrelieff." Machine Learning 2003.
    Zhao, Zheng et al. "On Similarity Preserving Feature Selection." TKDE 2013.
    """

    if "k" not in kwargs.keys():
        k = 5
    else:
        k = kwargs["k"]
    n_samples, n_features = X.shape

    # calculate pairwise distances between instances
    distance = pairwise_distances(X, metric='manhattan')

    score = np.zeros(n_features)

    # the number of sampled instances is equal to the number of total instances
    for idx in range(n_samples):
        near_hit = []
        near_miss = dict()

        self_fea = X[idx, :]
        c = np.unique(y).tolist()

        stop_dict = dict()
        for label in c:
            stop_dict[label] = 0
        del c[c.index(y[idx])]

        p_dict = dict()
        p_label_idx = float(len(y[y == y[idx]]))/float(n_samples)

        for label in c:
            p_label_c = float(len(y[y == label]))/float(n_samples)
            p_dict[label] = p_label_c/(1-p_label_idx)
            near_miss[label] = []

        distance_sort = []
        distance[idx, idx] = np.max(distance[idx, :])

        for i in range(n_samples):
            distance_sort.append([distance[idx, i], int(i), y[i]])
        distance_sort.sort(key=lambda x: x[0])

        for i in range(n_samples):
            # find k nearest hit points
            if distance_sort[i][2] == y[idx]:
                if len(near_hit) < k:
                    near_hit.append(distance_sort[i][1])
                elif len(near_hit) == k:
                    stop_dict[y[idx]] = 1
            else:
                # find k nearest miss points for each label
                if len(near_miss[distance_sort[i][2]]) < k:
                    near_miss[distance_sort[i][2]].append(distance_sort[i][1])
                else:
                    if len(near_miss[distance_sort[i][2]]) == k:
                        stop_dict[distance_sort[i][2]] = 1
            stop = True
            for (key, value) in stop_dict.items():
                    if value != 1:
                        stop = False
            if stop:
                break

        # update reliefF score
        near_hit_term = np.zeros(n_features)
        for ele in near_hit:
            near_hit_term = np.array(abs(self_fea-X[ele, :]))+np.array(near_hit_term)

        near_miss_term = dict()
        for (label, miss_list) in near_miss.items():
            near_miss_term[label] = np.zeros(n_features)
            for ele in miss_list:
                near_miss_term[label] = np.array(abs(self_fea-X[ele, :]))+np.array(near_miss_term[label])
            score += near_miss_term[label]/(k*p_dict[label])
        score -= near_hit_term/k
    return score

def feature_ranking(score):
    """
    Rank features in descending order according to reliefF score, the higher the reliefF score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [6]:
score = reliefF(X, y)
sorted = feature_ranking(score)

In [7]:
reliefF_sorted_features = []

for idx in sorted:
    reliefF_sorted_features.append(protein_code[idx])

In [8]:
print(reliefF_sorted_features)

['TTHY_HUMAN', 'ALBU_HUMAN', 'TRFE_HUMAN', 'APOE_HUMAN', 'CO3_HUMAN', 'HEMO_HUMAN', 'AACT_HUMAN', 'APOA1_HUMAN', 'PTGDS_HUMAN', 'ANGT_HUMAN', 'IGHG2_HUMAN', 'CLUS_HUMAN', 'APOH_HUMAN', 'CERU_HUMAN', 'CFAB_HUMAN', 'FINC_HUMAN', 'CH3L1_HUMAN', 'GELS_HUMAN', 'FETUA_HUMAN', 'ANT3_HUMAN', 'A1AG1_HUMAN', 'IC1_HUMAN', 'FBLN3_HUMAN', 'A2MG_HUMAN', 'KNG1_HUMAN', 'APOA4_HUMAN', 'RNAS1_HUMAN', 'ITIH4_HUMAN', 'C1S_HUMAN', 'DKK3_HUMAN', 'ENPP2_HUMAN', 'IGL1_HUMAN', 'CYTC_HUMAN', 'KLK6_HUMAN', 'FBLN1_HUMAN', 'HRG_HUMAN', 'PLTP_HUMAN', 'FBLN2_HUMAN', 'ZA2G_HUMAN', 'CFAI_HUMAN', 'C1QC_HUMAN', 'CNDP1_HUMAN', 'B4GA1_HUMAN', 'IBP7_HUMAN', 'TENX_HUMAN', 'HEP2_HUMAN', 'PEDF_HUMAN', 'CSTN1_HUMAN', 'SPRL1_HUMAN', 'CD166_HUMAN', 'NRX2A_HUMAN', 'ITIH2_HUMAN', 'A1BG_HUMAN', 'A1AG2_HUMAN', 'HPT_HUMAN', 'RET4_HUMAN', 'MYO6_HUMAN', 'NRP2_HUMAN', 'A2GL_HUMAN', 'APLP1_HUMAN', 'NID2_HUMAN', 'NPC2_HUMAN', 'LTBP2_HUMAN', 'APOD_HUMAN', 'CATD_HUMAN', 'CO4A_HUMAN', 'FIBA_HUMAN', 'CAH1_HUMAN', 'HBB_HUMAN', 'XYLT1_HUMAN', '

# Chi2

In [9]:
import numpy as np
from sklearn.feature_selection import chi2


def chi_square(X, y):
    """
    This function implements the chi-square feature selection (existing method for classification in scikit-learn)

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array},shape (n_samples,)
        input class labels

    Output
    ------
    F: {numpy array}, shape (n_features,)
        chi-square score for each feature
    """
    F, pval = chi2(X, y)
    return F


def feature_ranking(F):
    """
    Rank features in descending order according to chi2-score, the higher the chi2-score, the more important the feature is
    """
    idx = np.argsort(F)
    return idx[::-1]

In [10]:
F = chi_square(X, y)
sorted = feature_ranking(F)

In [11]:
chi2_sorted_features = []

for idx in sorted:
    chi2_sorted_features.append(protein_code[idx])

In [12]:
print(chi2_sorted_features)

['TTHY_HUMAN', 'TRFE_HUMAN', 'ALBU_HUMAN', 'CO3_HUMAN', 'IGHG2_HUMAN', 'HPT_HUMAN', 'ANGT_HUMAN', 'AACT_HUMAN', 'APOA1_HUMAN', 'FINC_HUMAN', 'FETUA_HUMAN', 'A1AG1_HUMAN', 'CH3L1_HUMAN', 'HEMO_HUMAN', 'IGL1_HUMAN', 'ANT3_HUMAN', 'APOH_HUMAN', 'OMGP_HUMAN', 'NRX2A_HUMAN', 'GELS_HUMAN', 'APOA4_HUMAN', 'FBLN2_HUMAN', 'FBLN3_HUMAN', 'CNDP1_HUMAN', 'CD166_HUMAN', 'CYTC_HUMAN', 'ENPP2_HUMAN', 'ZA2G_HUMAN', 'A2MG_HUMAN', 'A1BG_HUMAN', 'GAS6_HUMAN', 'PLTP_HUMAN', 'DKK3_HUMAN', 'C1S_HUMAN', 'HRG_HUMAN', 'CLUS_HUMAN', 'IBP7_HUMAN', 'IGG1_HUMAN', 'C1QC_HUMAN', 'PGS2_HUMAN', 'CAH1_HUMAN', 'KLK6_HUMAN', 'A2GL_HUMAN', 'CAD13_HUMAN', 'A1AT_HUMAN', 'HBA_HUMAN', 'NRP2_HUMAN', 'DPP6_HUMAN', 'RET4_HUMAN', 'B4GA1_HUMAN', 'IC1_HUMAN', 'SODE_HUMAN', 'CRAC1_HUMAN', 'KNG1_HUMAN', 'ITIH4_HUMAN', 'CSTN1_HUMAN', 'CFAI_HUMAN', 'LRP1_HUMAN', 'SEM3G_HUMAN', 'HBB_HUMAN', 'NPC2_HUMAN', 'NID2_HUMAN', 'NRX1A_HUMAN', 'APLP1_HUMAN', 'FIBA_HUMAN', 'KVD37_HUMAN', 'APOB_HUMAN', 'AATM_HUMAN', 'AFAM_HUMAN', 'RNAS1_HUMAN', 'APO

# Minimum redundancy Maximum Relevance

In [45]:
from skfeature.function.information_theoretical_based import LCSI


def mrmr(X, y, **kwargs):
    """
    This function implements the MRMR feature selection

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data, guaranteed to be discrete
    y: {numpy array}, shape (n_samples,)
        input class labels
    kwargs: {dictionary}
        n_selected_features: {int}
            number of features to select

    Output
    ------
    F: {numpy array}, shape (n_features,)
        index of selected features, F[0] is the most important feature
    J_CMI: {numpy array}, shape: (n_features,)
        corresponding objective function value of selected features
    MIfy: {numpy array}, shape: (n_features,)
        corresponding mutual information between selected features and response

    Reference
    ---------
    Brown, Gavin et al. "Conditional Likelihood Maximisation: A Unifying Framework for Information Theoretic Feature Selection." JMLR 2012.
    """
    if 'n_selected_features' in kwargs.keys():
        n_selected_features = kwargs['n_selected_features']
        F, J_CMI, MIfy = LCSI.lcsi(X, y, gamma=0, function_name='MRMR', n_selected_features=n_selected_features)
    else:
        F, J_CMI, MIfy = LCSI.lcsi(X, y, gamma=0, function_name='MRMR')
    return F, J_CMI, MIfy

In [47]:
F, J_CMI, MIfy = mrmr(X, y)

[  0 459]


# Filter methods

## Information Gain

In [53]:
from sklearn.feature_selection import mutual_info_classif

importances = mutual_info_classif(X, y)
feat_importances = pd.Series(importances, lisbon_coimbra_df.columns[0:len(lisbon_coimbra_df.columns)])

feat_importances = feat_importances.sort_values(ascending=False)

In [54]:
print(feat_importances)

FETUA_HUMAN    0.213672
FBLN2_HUMAN    0.212962
IBP7_HUMAN     0.211470
CFAI_HUMAN     0.206867
CD109_HUMAN    0.200493
                 ...   
FRS1L_HUMAN    0.000000
KV106_HUMAN    0.000000
PRIO_HUMAN     0.000000
SHPS1_HUMAN    0.000000
ALBU_HUMAN     0.000000
Length: 517, dtype: float64


## Fisher's Score

In [58]:
from skfeature.function.similarity_based import fisher_score

ranks = fisher_score.fisher_score(X, y)

feat_sorted_fisher = []
for idx in ranks:
    feat_sorted_fisher.append(protein_code[idx])

# ANOVA f-test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from matplotlib import pyplot
 
fs = SelectKBest(score_func=f_classif, k='all')
# learn relationship from training data
fs.fit(X, y)
# transform train input data
X_train_fs = fs.transform(X_train)
# transform test input data
X_test_fs = fs.transform(X_test)
return X_train_fs, X_test_fs, fs
 
## split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# feature selection
X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test)
# what are scores for the features
for i in range(len(fs.scores_)):
	print('Feature %d: %f' % (i, fs.scores_[i]))
# plot the scores
pyplot.bar([i for i in range(len(fs.scores_))], fs.scores_)
pyplot.show()